In [46]:
#
import random
import math
import hashlib

import argparse
import numpy as np


from logging import getLogger, StreamHandler, DEBUG

logger = getLogger("root")
handler = StreamHandler()
handler.setLevel(DEBUG)
logger.setLevel(DEBUG)
logger.addHandler(handler)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.propagate = False



In [54]:

logger.warning("test")
logger.info("info test")
logger.debug("debug test")

2018-02-02 11:25:32,096 - root - WARNING - test
2018-02-02 11:25:32,096 - root - WARNING - test
2018-02-02 11:25:32,100 - root - INFO - info test
2018-02-02 11:25:32,100 - root - INFO - info test
2018-02-02 11:25:32,104 - root - DEBUG - debug test
2018-02-02 11:25:32,104 - root - DEBUG - debug test


In [52]:
def logTest():
    logger = getLogger("logtest")
    logger.setLevel(DEBUG)
    logger.warning("test")
    logger.info("info test")
    logger.debug("debug test")    

In [53]:
logTest()

INFO:logtest:info test
DEBUG:logtest:debug test


In [50]:
root = logger
map(root.removeHandler, root.handlers[:])
map(root.removeFilter, root.filters[:])


In [139]:
class Board(object):
    def __init__(self):
        self.board = np.array( [" "] * 10 )
        
    def update(self,move,piece):
        self.board[move] = piece

    def isAvailable(self,move):
        s = self.isfilled()
        if move in s:
            return True
        else:
            return False
        
    def isfilled(self):
        space = np.where(self.board == " ")[0]
        return space[1:]

    def board_turn(self):
        space = self.isfilled()
        print("** available space ..", space)
        if len(space) > 0:
            turn = 9- len(space)
        if len(space) == 0:
            turn = 9
        return turn
    
    def __repr__(self):
        print('   |   |')
        print(' ' + self.board[7] + ' | ' + self.board[8] + ' | ' + self.board[9])
        print('   |   |')
        print('-----------')
        print('   |   |')
        print(' ' + self.board[4] + ' | ' + self.board[5] + ' | ' + self.board[6])
        print('   |   |')
        print('-----------')
        print('   |   |')
        print(' ' + self.board[1] + ' | ' + self.board[2] + ' | ' + self.board[3])
        print('   |   |')

        space = self.isfilled()
        print("** available space ..", space)
        if len(space) > 0:
            self.turn = 9- len(space)
        if len(space) == 0:
            self.turn = 9
        s="CurrentState ...: turn %d"%(self.turn)
        return s
            
    def isWin(self,le):
        # bo - input board style
        # le - letter on board from player's piece
        bo = self.board.copy()
        return ((bo[7] == le and bo[8] == le and bo[9] == le) or # across the top
        (bo[4] == le and bo[5] == le and bo[6] == le) or # across the middle
        (bo[1] == le and bo[2] == le and bo[3] == le) or # across the bottom
        (bo[7] == le and bo[4] == le and bo[1] == le) or # down the left side
        (bo[8] == le and bo[5] == le and bo[2] == le) or # down the middle
        (bo[9] == le and bo[6] == le and bo[3] == le) or # down the right side
        (bo[7] == le and bo[5] == le and bo[3] == le) or # diagonal
        (bo[9] == le and bo[5] == le and bo[1] == le)) # diagonal
        


In [188]:
class GameState2():

    NUM_TURNS = 10
    GOAL = 0
    PIECE=["O","X"]

    num_moves=10

    def __init__(self, board=Board(), turn=0, next_turn=0,test=False):
        self.board=board
        self.turn=turn
        self.next_turn=next_turn
        
        if test:
            print(self.board)
            print("turn : next_turn",turn,next_turn)

    def first_turn(self):
        self.first_turn = np.random.choice([0,1])
        if self.first_turn:
            print("** Human first..")
            self.next_turn = 0
        else:
            print("* AI first..")
            self.next_turn = 1

    def next_state(self):

        sp = self.board.isfilled()
        move = np.random.choice(sp)
        if self.next_turn:
            print("AI turn...")
            self.board.update(move,self.PIECE[0])
            self.next_turn = 0
        else:
            print("Human turn...")
            self.board.update(move,self.PIECE[1])
            self.next_turn = 1
            
        turn = self.board.board_turn()
        next=GameState2(board= self.board,turn= turn, next_turn = self.next_turn, test=True)
        return next

    def terminal(self):
        if self.turn == 9:
            return True
        
        ai_piece = self.PIECE[0]
        hu_piece = self.PIECE[1]
        if self.board.isWin(ai_piece) or self.board.isWin(hu_piece):
            return True
        
        return False

    def reward(self):
        #r = 1.0-(abs(self.value-self.GOAL)/self.MAX_VALUE)
        ai_piece = self.PIECE[0]
        hu_piece = self.PIECE[1]

        if self.board.isWin(ai_piece):
            return 10
        elif self.board.isWin(hu_piece):
            return -10
        
        return 0

    def __hash__(self):
        return int(hashlib.md5( str( self.board.board) ).hexdigest(),16)

    def __eq__(self,other):
        if hash(self)==hash(other):
            return True
        return False

    def __repr__(self):
        s="Turn: %d; Next Turn: %s"%(self.turn,self.next_turn)
        return s


In [200]:
def TREEPOLICY(node):
    #a hack to force 'exploitation' in a game where there are many options, and you may never/not want to fully expand first
    while node.state.terminal()==False:
        if len(node.children)==0:
            return EXPAND(node)
        elif random.uniform(0,1)<.5:
            node=BESTCHILD(node,SCALAR)
        else:
            if node.fully_expanded()==False:
                return EXPAND(node)
            else:
                node=BESTCHILD(node,SCALAR)
    return node

def EXPAND(node):
    tried_children=[c.state for c in node.children]
    new_state=node.state.next_state()
    while new_state in tried_children:
        new_state=node.state.next_state()
    node.add_child(new_state)
    return node.children[-1]

#current this uses the most vanilla MCTS formula it is worth experimenting with THRESHOLD ASCENT (TAGS)
def BESTCHILD(node,scalar):
    bestscore=0.0
    bestchildren=[]
    for c in node.children:
        exploit=c.reward/c.visits
        explore=math.sqrt(2.0*math.log(node.visits)/float(c.visits))
        score=exploit+scalar*explore
        if score==bestscore:
            bestchildren.append(c)
        if score>bestscore:
            bestchildren=[c]
            bestscore=score
    if len(bestchildren)==0:
        logger.warn("OOPS: no best child found, probably fatal")
    return random.choice(bestchildren)

def DEFAULTPOLICY(state):
    while state.terminal()==False:
        state=state.next_state()
    return state.reward()

def BACKUP(node,reward):
    while node!=None:
        node.visits+=1
        node.reward+=reward
        node=node.parent
    return


In [161]:
class Node():

    def __init__(self, state, parent=None):
        self.visits=1
        self.reward=0.0
        self.state=state
        self.children=[]
        self.parent=parent

    def add_child(self,child_state):
        child=Node(child_state,self)
        self.children.append(child)
    def update(self,reward):
        self.reward+=reward
        self.visits+=1
    def fully_expanded(self):
        if len(self.children)==self.state.num_moves:
            return True
        return False
    def __repr__(self):
        s="Node; children: %d; visits: %d; reward: %f"%(len(self.children),self.visits,self.reward)
        return s



In [189]:
root = Node(GameState2(board=Board(),turn=0, next_turn=0,test=True))

   |   |
   |   |  
   |   |
-----------
   |   |
   |   |  
   |   |
-----------
   |   |
   |   |  
   |   |
** available space .. [1 2 3 4 5 6 7 8 9]
CurrentState ...: turn 0
turn : next_turn 0 0


In [190]:
tree_node = TREEPOLICY(root)

Human turn...
** available space .. [1 2 3 5 6 7 8 9]
   |   |
   |   |  
   |   |
-----------
   |   |
 X |   |  
   |   |
-----------
   |   |
   |   |  
   |   |
** available space .. [1 2 3 5 6 7 8 9]
CurrentState ...: turn 1
turn : next_turn 1 1


In [191]:
DEFAULTPOLICY( tree_node.state )

AI turn...
** available space .. [1 2 3 5 6 7 9]
   |   |
   | O |  
   |   |
-----------
   |   |
 X |   |  
   |   |
-----------
   |   |
   |   |  
   |   |
** available space .. [1 2 3 5 6 7 9]
CurrentState ...: turn 2
turn : next_turn 2 0
Human turn...
** available space .. [2 3 5 6 7 9]
   |   |
   | O |  
   |   |
-----------
   |   |
 X |   |  
   |   |
-----------
   |   |
 X |   |  
   |   |
** available space .. [2 3 5 6 7 9]
CurrentState ...: turn 3
turn : next_turn 3 1
AI turn...
** available space .. [2 3 6 7 9]
   |   |
   | O |  
   |   |
-----------
   |   |
 X | O |  
   |   |
-----------
   |   |
 X |   |  
   |   |
** available space .. [2 3 6 7 9]
CurrentState ...: turn 4
turn : next_turn 4 0
Human turn...
** available space .. [3 6 7 9]
   |   |
   | O |  
   |   |
-----------
   |   |
 X | O |  
   |   |
-----------
   |   |
 X | X |  
   |   |
** available space .. [3 6 7 9]
CurrentState ...: turn 5
turn : next_turn 5 1
AI turn...
** available space .. [3 7 9]
 

-10

In [192]:
tree_node

Node; children: 0; visits: 1; reward: 0.000000

In [193]:
BACKUP(tree_node,-10)

In [194]:
tree_node

Node; children: 0; visits: 2; reward: -10.000000

In [196]:
len(root.children)

1

In [197]:
root.children[0].reward

-10.0

In [201]:
BESTCHILD(root,0)

/Users/donchan/anaconda3/envs/tensorflow3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead


IndexError: Cannot choose from an empty sequence